# Baseline (01)

# Imports

In [1]:
!pip install "git+http://github.com/snowdd1/wcs.git" >/dev/null

  Running command git clone -q http://github.com/snowdd1/wcs.git /tmp/pip-req-build-9r5i92j3


In [2]:
import pandas as pd
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from wcs.google import google_drive_share

import warnings
warnings.simplefilter(action='ignore')

# Load Data

In [3]:
data_dir_url = 'https://drive.google.com/file/d/1etygmCq28-9yWBhlFwXvFqPkcBKaq8Ow/view?usp=sharing'  # zip

DATA_DIR = './data'

In [4]:
if not os.path.exists(DATA_DIR):
  with open('data_dir.zip', 'wb') as f:
    f.write( google_drive_share(data_dir_url, binaryfile=True).read() )

  !unzip data_dir.zip > /dev/null

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: data/img/zjhVHDCCl2nwR1yCPZX4xw819Se.jpg  
  inflating: data/img/wmMq5ypRNJbWpdhC9aPjpdx1MMp.jpg  
  inflating: data/img/6NwKjz1FyPNZK6j4OBzpzohcvvF.jpg  
  inflating: data/img/i8lVYRplWdCEqYOutxpxeCtaD2a.jpg  
  inflating: data/img/zt2ZJfsE1OOFkuwy6bgatajn23o.jpg  
  inflating: data/img/nMNVfz2bmcgj9Du0wywVHjbRIsz.jpg  
  inflating: data/img/r9gEJ6BrgcDoRpA8dByGcpK1ScN.jpg  
  inflating: data/img/zCDr1NvXgfhlSeUpJZ5vr5NLn2z.jpg  
  inflating: data/img/NHyU3VkWyTVufyP6Gu4SCHPEiy.jpg  
  inflating: data/img/8lI1p5cPqgXN2qrKZrmI3mhKBfs.jpg  
  inflating: data/img/mro2I5vlcwn2XHwgSDwk1DvhanM.jpg  
  inflating: data/img/fTze8m40Zqpg64Amth7VUOhlCDs.jpg  
  inflating: data/img/zSINB8kT5uBArawlQljdl6WIk7F.jpg  
  inflating: data/img/k2WWlrMbuINyzDCfBuD48eN22QY.jpg  
  inflating: data/img/9CHg7tCS8oPdKszDUWw5RnDLKrC.jpg  
  inflating: data/img/gBhLQmpCPoKFMCGsulMbIFzrBID.jpg  
  inflating: data/img/nZVOeGCbhLzIfCOB

# Preproc

In [5]:
df = pd.read_csv('data/movies.csv')

In [6]:
# use only mot null rows
df = df.dropna()
# set data path to basename of the file
df['data_path'] = df['poster_path'].apply(lambda x: x.split('/')[-1])
# remove rows with empty genre_id list and set correct list type
df['genre_ids'] = df['genre_ids'].apply(lambda x: np.nan if len(eval(x)) == 0 else x)
df = df.dropna()
df['genre_ids'] = df['genre_ids'].apply(lambda x: eval(x))

print(f'len of df: {len(df)}')

len of df: 9878


 Create ImageGenerators

In [7]:
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='./data/img',
    x_col="data_path",
    y_col="genre_ids",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training'
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='./data/img',
    x_col="data_path",
    y_col="genre_ids",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='training'
)

Found 8891 validated image filenames belonging to 19 classes.
Found 8891 validated image filenames belonging to 19 classes.


In [8]:
# show class indicies
print(train_generator.class_indices)
print('length:', len(train_generator.class_indices))

{12: 0, 14: 1, 16: 2, 18: 3, 27: 4, 28: 5, 35: 6, 36: 7, 37: 8, 53: 9, 80: 10, 99: 11, 878: 12, 9648: 13, 10402: 14, 10749: 15, 10751: 16, 10752: 17, 10770: 18}
length: 19


# Simple Model

In [9]:
model = keras.Sequential(
    [
      layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
      layers.Activation('relu'),
      layers.Conv2D(32, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Conv2D(64, (3, 3), padding='same'),
      layers.Activation('relu'),
      layers.Conv2D(64, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Flatten(),
      layers.Dense(512),
      layers.Activation('relu'),
      layers.Dropout(0.5),
      layers.Dense(len(train_generator.class_indices), activation='sigmoid')
    ]
)


model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

# Train

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10
)

Epoch 1/10
 17/278 [>.............................] - ETA: 8:07 - loss: 1.4935 - accuracy: 0.0735